In [1]:
%pip install openpyxl "psycopg[binary,pool]" pytz



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Load Spreadsheet into a Python data structure for processing

In [2]:
CONSUMPTION = "CONSUMPTION"
GENERATION = "GENERATION"


FILE_NAME = "/home/martin/Downloads/RC101533-Energy-Report-20241123_20241130 (1).xlsx"


In [3]:
from openpyxl import load_workbook
from datetime import datetime


class Measurement:
    def __init__(self, period_start, period_end, meter_code, meter_point):
        self.data_list = []
        self.data_map = {}
        self.period_start = period_start
        self.period_end = period_end
        self.meter_code = meter_code
        self.meter_point = meter_point

    def add_value_at_timestamp(self, value, timestamp):
        list_item = {
            'value': value,
            'timestamp': timestamp
        }
        if not list_item in self.data_list:
            self.data_list.append(list_item)
        
        self.data_map[timestamp] = value
        
    def slice_by_timestamp(self, start, end):
        slice = Measurement(start, end, self.meter_code, self.meter_point)
        for list_item in self.data_list:
            if start <= list_item['timestamp'] <= end:
                slice.add_value_at_timestamp(list_item['value'], list_item['timestamp'])
        return slice
    
    def sum(self):
        sum = 0.0
        for list_item in self.data_list:
            sum += list_item['value']
        return sum
    
    def get_x_axis(self):
        return list(map(lambda x: x['timestamp'], self.data_list))
    
    def get_y_axis(self):
        return list(map(lambda x: x['value'], self.data_list))
    
    def to_string(self):
        return f"{self.meter_code}"
        
    
        
class MeterPoint:
    def __init__(self, point_id, energy_direction, customer_name):
        self.measurements = []
        self.id = point_id
        self.energy_direction = energy_direction
        self.customer_name = customer_name
        
    def add_measurement(self, period_start, period_end, meter_code):
        measurement = Measurement(period_start, period_end, meter_code, self)
        self.measurements.append(measurement)
        return measurement
    
    def get_measurement_by_meter_code(self, meter_code):
        for measurement in self.measurements:
            if measurement.meter_code == meter_code:
                return measurement
        return None
        
    def to_string(self):
        return f"{self.id} {self.energy_direction} {self.customer_name}"
    
        

class EnergyData:
    def __init__(self, filename):
        self.timestamps = []
        self.meter_points = {}
        self.calculated_totals = []
        self.filename = filename
        self._load_file(filename)
        self._calculate_totals()
        
    def _load_file(self, filename):
        wb = load_workbook(filename)
        work_sheet = wb['Energiedaten']
        # first row empty
        assert(work_sheet['A1'].value is None)
        assert(work_sheet['A8'].value is None)
        assert(work_sheet['A9'].value is None)
        assert(work_sheet['A10'].value is None)
        assert(work_sheet['A11'].value is not None)
        self._parse_worksheet(work_sheet)
        
    
    def _add_meter_point(self, point_id, energy_direction, customer_name):
        if not point_id in self.meter_points:
            self.meter_points[point_id] = MeterPoint(point_id, energy_direction, customer_name)
        return self.meter_points[point_id]
    
    
    def _parse_timeline(self, work_sheet):
        row = 11
        while True:
            v = work_sheet.cell(row=row, column=1).value
            if v is None:
                break
            else:
                self.timestamps.append(datetime.strptime(v, "%d.%m.%Y %H:%M:%S"))
            row = row + 1
    
    
    def _parse_worksheet(self, work_sheet):
        # timestamps first
        self._parse_timeline(work_sheet)
        
        # all data cols for each meter_point
        i = 2
        while True:
            # point ID is in row 2
            cell = work_sheet.cell(row=2, column=i)
            point_id = cell.value
            
            if point_id is None:
                break
            else:
                # row and column index of the current point ID for offset reference
                r = cell.row
                c = cell.column
                
                energy_direction = work_sheet.cell(row=r+2, column=c).value
                customer_name = work_sheet.cell(row=r+1, column=c).value
                meter_point = self._add_meter_point(point_id, energy_direction, customer_name)
                
                measurement = meter_point.add_measurement(
                    period_start = datetime.strptime(work_sheet.cell(row=r+3, column=c).value, "%d.%m.%Y %H:%M:%S"),
                    period_end = datetime.strptime(work_sheet.cell(row=r+4, column=c).value, "%d.%m.%Y %H:%M:%S"),
                    meter_code = work_sheet.cell(row=r+5, column=c).value,  # name of the metric
                )
                
                data_start_row = r+9 # row 11
                # iterate rows til hitting bottom
                while True:
                    # only fields between period_start and period_end are valid
                    # get timestamp from column 1 of current row
                    v = work_sheet.cell(row=data_start_row, column=1).value
                    if v is None:
                        # end of timeline
                        break
                    
                    timestamp = datetime.strptime(v, "%d.%m.%Y %H:%M:%S")
                    
                    if not measurement.period_start <= timestamp <= measurement.period_end:
                        #ignore field
                        #print("Ignoring", meter_point.id, timestamp)
                        data_start_row = data_start_row + 1
                        continue
                    
                    # actual value
                    v = work_sheet.cell(row=data_start_row, column=c).value                    
                    if v is None:
                        # redundant
                        break
                    else:
                        if isinstance(v, float):
                            measurement.add_value_at_timestamp(v, timestamp)
                        else:
                            #print(f"Warning: no value at {timestamp} for {measurement.to_string()}")
                            measurement.add_value_at_timestamp(0.0, timestamp)
                        
                    data_start_row = data_start_row + 1
                # end loop
            i = i + 1
        #end loop
        
            
    # returns a lists of metrics
    def get_metrics(self):
        metrics = []
        
        for key in self.meter_points:
            #print(key)
            meter_point = self.meter_points[key]
            for measurement in meter_point.measurements:
                metric = {
                    'energy_direction': meter_point.energy_direction,
                    'meter_code': measurement.meter_code
                }
                if metric not in metrics:
                    metrics.append(metric)

        return metrics
            
        
    def print_metrics(self):
        metrics = self.get_metrics()
        for metric in metrics:
            print(metric)
                
                
    def get_measurements_by_metric(self, metric, src_measurements=None):
        measurements = []
        if src_measurements is None:
            # take all measurements from energy_data
            for key in self.meter_points:
                meter_point = self.meter_points[key]
                for measurement in meter_point.measurements:
                    if measurement.meter_code == metric['meter_code'] and meter_point.energy_direction == metric['energy_direction']:
                        measurements.append(measurement)
        else:
            # filter src_measurements
            for measurement in src_measurements:
                    if measurement.meter_code == metric['meter_code']:
                        measurements.append(measurement)
            
        return measurements
    
    
    def get_measurements_by_customer(self, customer_name):
        measurements = []
        
        for key in self.meter_points:
            meter_point = self.meter_points[key]
            if meter_point.customer_name == customer_name:
                for measurement in meter_point.measurements:
                    measurements.append(measurement)
        
        return measurements
    
                
    # for each pair (energy_direction, meter_code) we keep 
    # a Measurement of the Sums of all points
    def _calculate_totals(self):
        metrics = self.get_metrics()
        
        for metric in metrics:
            #print(metric)
            measurements = self.get_measurements_by_metric(metric)
            
            #for measurement in measurements:
            #    print(f"\t{measurement.meter_point.id}")
            
            summed_measurement = Measurement(
                self.timestamps[0], 
                self.timestamps[-1], 
                f"Sum ({metric['energy_direction']}) of {metric['meter_code']}",
                meter_point=None
            )
            for timestamp in self.timestamps:
                
                sum = 0.0
                for measurement in measurements:
                    # only values we actually have
                    if timestamp in measurement.data_map:
                        value = measurement.data_map[timestamp]
                        sum += value
                
                #print(sum, timestamp)
                summed_measurement.add_value_at_timestamp(sum, timestamp)
            
            
            self.calculated_totals.append(summed_measurement)
        
        return None
        
    
    # sum over all measurement points having the same metric    
    def sum_measurements_by_metric(self, metric):
        measurements = self.get_measurements_by_metric(metric)
        sum_of_consumers = Measurement(energy_data.timestamps[0], energy_data.timestamps[-1], f"Summe {metric['meter_code']}", None)

        for timestamp in energy_data.timestamps:
            sum = 0.0
            for measurement in measurements:
                if timestamp in measurement.data_map:
                    sum += measurement.data_map[timestamp]
            sum_of_consumers.add_value_at_timestamp(sum, timestamp)
        
        return sum_of_consumers
    
    
    # sum of metric grouped by time over all days
    def dayly_average_sum_by_metric(self, metric, measurements=None):
        if measurements is None:
            measurements = self.get_measurements_by_metric(metric)
        else:
            # filter parameter measurements by metric
            measurements = self.get_measurements_by_metric(metric, measurements)

        
        #timeline of a single day
        a_day = []
        for hour in range(0, 24):
            for quarter in ["00", "15", "30", "45"]:
                d = datetime.strptime(f"{hour}:{quarter}:00", "%H:%M:%S")
                a_day.append(d)
                
        average = Measurement(a_day[0], a_day[-1], f"Summe {metric['meter_code']}", None)
        
        for timestamp in a_day:
            time = timestamp.time()
            #print("time:", time)
            sum = 0.0
            #count = 0
            for measurement in measurements:
                for value in measurement.data_list:
                    t = value['timestamp'].time()
                    if t == time:
                        #print(t)
                        sum += value['value']
                        #count += 1
            avg = sum # / count
            average.add_value_at_timestamp(avg, timestamp)
            
        return average



energy_data = EnergyData(FILE_NAME)
#energy_data.print_metrics()

In [4]:
import psycopg
import os
import pytz
import time

os.environ['PGSERVICEFILE'] = "/home/martin/Workspace/Energiegemeinschaft/notebooks/.pg_service.conf"
os.environ['PGPASSFILE'] = "/home/martin/Workspace/Energiegemeinschaft/notebooks/.pgpass"

with psycopg.connect(service='eeg-middleware') as conn:
    with conn.cursor() as cur:
        
        # iterate measurement points
        for key in energy_data.meter_points:
            point = energy_data.meter_points[key]
            print(time.time(), point.id)
            print(time.time(), point.customer_name)
            
            # fetch measurement point id from db
            cur.execute("""
                SELECT id from members_measurementpoint
                WHERE identifier like %s
            """, (point.id,))
            rows = cur.fetchall()
            if len(rows) > 1:
                raise Exception(f"Duplicate IDs for name: {point.customer_name}")
            m_point_id = rows[0][0]
            print(time.time(), "m_point_id", m_point_id)
            
            # iterate measurements of measurement point
            
            for measurement in point.measurements:
                num_vals = len(measurement.data_list)
                print(time.time(), f"    {num_vals} {measurement.meter_code}")
    
                #extract unit and description
                unit = measurement.meter_code.split(" ")[-1].replace("[", "").replace("]", "")
                description = " ".join(measurement.meter_code.split(" ")[:-1])
                
                # make sure all meter_codes are in db
                q = """
                    INSERT INTO metering_metercode (id, description, unit)
                    select nextval('metering_metercode_id_seq'), %s, %s
                    where not exists (
                        select 1 from metering_metercode 
                        where description like %s
                        and unit like %s
                    );
                """
                cur.execute(q, (description, unit, description, unit))
                
                # get meter_code id
                cur.execute("""
                    SELECT id from metering_metercode
                    where description like %s
                        and unit like %s
                """, (description, unit))
                rows = cur.fetchall()
                if len(rows) > 1:
                    raise Exception(f"Duplicate IDs for description: {description}")
                meter_code_id = rows[0][0]
                print(time.time(), "meter_code_id", meter_code_id)
                
                
                for val in measurement.data_list:
                    #print(time.time(), val['value'], val['timestamp'])
                    
                    ## adding the correct time zone before inserting into db
                    ts_without_timezone = val['timestamp']
                    timezone = pytz.timezone("Europe/Vienna")
                    ts_with_timezone = timezone.localize(ts_without_timezone)
                    
                    
                    q = """
                        INSERT INTO metering_measurement (id, timestamp, value, measurement_point_id, meter_code_id)
                        select nextval('metering_measurement_id_seq'), %s, %s, %s, %s
                        ON CONFLICT (timestamp, measurement_point_id, meter_code_id)
                        DO UPDATE SET
                            value = EXCLUDED.value
                        ;
                    """
                    cur.execute(q, (ts_with_timezone, val['value'], m_point_id, meter_code_id))
                    
                    
        #            list_item = {
        #    'value': value,
        #    'timestamp': timestamp
        #}
                    
            
            

        conn.commit()

    

1733424766.4113092 AT0030000000000000000000000118026
1733424766.411352 Markus Hilpert
1733424766.4679544 m_point_id 27
1733424766.4679701     768 Gesamtverbrauch lt. Messung (bei Teilnahme gem. Erzeugung) [KWH]
1733424766.5269682 meter_code_id 193
1733424788.4998755     768 Anteil gemeinschaftliche Erzeugung [KWH]
1733424788.5567129 meter_code_id 194
1733424811.1399217     768 Eigendeckung gemeinschaftliche Erzeugung [KWH]
1733424811.1963933 meter_code_id 195
1733424832.7791817 AT0030000000000000000000000122367
1733424832.779251 Nicole Aichinger
1733424832.8071804 m_point_id 79336
1733424832.8072035     768 Gesamtverbrauch lt. Messung (bei Teilnahme gem. Erzeugung) [KWH]
1733424832.863836 meter_code_id 193
1733424854.6801147     768 Anteil gemeinschaftliche Erzeugung [KWH]
1733424854.7456064 meter_code_id 194
1733424877.1762788     768 Eigendeckung gemeinschaftliche Erzeugung [KWH]
1733424877.2886963 meter_code_id 195
1733424898.8612564 AT0030000000000000000000000396479
1733424898.8613